<a href="https://colab.research.google.com/github/rishank012/Capstone-Project/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
import os

# Step 1: Importing and Preprocessing the Data

In [ ]:
    from google.colab import files
    uploaded = files.upload()

In [ ]:
df = pd.read_csv('dataset.csv')
df

In [ ]:
num_cols = [
    'ID',                  # parking lot identifier
    'Occupancy',
    'Capacity',
    'QueueLength',
    'TrafficConditionNearby',
    'IsSpecialDay'
]
df[num_cols] = (
    df[num_cols]
      .apply(lambda col: pd.to_numeric(col, errors='coerce'))
      .fillna(0)
)

In [ ]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                 format='%d-%m-%Y %H:%M:%S')
df = df.sort_values(['ID', 'Timestamp']).reset_index(drop=True)

# Feature Engineering:

In [ ]:
df['occ_rate'] = df['Occupancy'] / df['Capacity']

for col in ['QueueLength', 'TrafficConditionNearby']:
    mn = df.groupby('ID')[col].transform('min')
    mx = df.groupby('ID')[col].transform('max')
    df[f'{col}_norm'] = (df[col] - mn) / (mx - mn + 1e-6)

vt_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df['VT_weight'] = df['VehicleType'].map(vt_map).fillna(1.0)

### **Model 1:** *Baseline Linear Model*

In [ ]:
alpha = 0.5
df['delta1'] = alpha * df['occ_rate']
df['price_m1'] = df.groupby('ID')['delta1'].cumsum().add(10)

### **Model 2:** *Demand Based Price Function*

In [ ]:
a, b, c, d, e = 1.0, 0.8, 0.6, 1.2, 0.5
lam = 0.8

df['raw_d'] = (
      a * df['occ_rate']
    + b * df['QueueLength_norm']
    - c * df['TrafficConditionNearby_norm']
    + d * df['IsSpecialDay']
    + e * df['VT_weight']
)

d_min = df.groupby('ID')['raw_d'].transform('min')
d_max = df.groupby('ID')['raw_d'].transform('max')
df['d_norm'] = (df['raw_d'] - d_min) / (d_max - d_min + 1e-6)

df['price_m2'] = 10 * (1 + lam * df['d_norm'])
df['price_m2'] = df['price_m2'].clip(lower=5, upper=20)
a, b, c, d, e = 1.0, 0.8, 0.6, 1.2, 0.5
lam = 0.8

df['raw_d'] = (
      a * df['occ_rate']
    + b * df['QueueLength_norm']
    - c * df['TrafficConditionNearby_norm']
    + d * df['IsSpecialDay']
    + e * df['VT_weight']
)

d_min = df.groupby('ID')['raw_d'].transform('min')
d_max = df.groupby('ID')['raw_d'].transform('max')
df['d_norm'] = (df['raw_d'] - d_min) / (d_max - d_min + 1e-6)

df['price_m2'] = 10 * (1 + lam * df['d_norm'])
df['price_m2'] = df['price_m2'].clip(lower=5, upper=20)

###**Model 3:** *Competitive Pricing Model*

In [ ]:
def comp_logic(row):
    if row['Occupancy'] >= row['Capacity']:
        row['price_m3'] = row['price_m2'] * 0.7
        row['suggest_reroute'] = True
    else:
        uplift = 0.1 * row['price_m2'] * (1 - row['TrafficConditionNearby_norm'])
        row['price_m3'] = row['price_m2'] + uplift
        row['suggest_reroute'] = False
    return row

df = df.apply(comp_logic, axis=1)

In [ ]:
df.to_csv('parking.csv', index=False)

### **Defining Pathway Schema:**

In [ ]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    ID: int
    Occupancy: int
    Capacity: int
    occ_rate: float
    QueueLength_norm: float
    TrafficConditionNearby_norm: float
    IsSpecialDay: int
    VT_weight: float
    price_m1: float
    price_m2: float
    price_m3: float
    suggest_reroute: bool

In [ ]:
data = pw.demo.replay_csv(
    'parking.csv',
    schema=ParkingSchema,
    input_rate=500
)

In [ ]:
fmt = '%Y-%m-%d %H:%M:%S'
stream = data.with_columns(t = data.Timestamp.dt.strptime(fmt))

## **Bokeh visualization of all three models:**

In [ ]:
def multi_price_plot(source):
    fig = bokeh.plotting.figure(
        height=400, width=900,
        title='Dynamic Parking Prices: Models 1, 2 & 3',
        x_axis_type='datetime'
    )
    fig.line('t','price_m1', source=source, color='blue',  legend_label='Model 1')
    fig.line('t','price_m2', source=source, color='green',legend_label='Model 2')
    fig.line('t','price_m3', source=source, color='red',   legend_label='Model 3')
    fig.legend.location = 'top_left'
    return fig

pn.extension()
viz = stream.plot(multi_price_plot, sorting_col='t')
pn.Column(viz).servable()

In [ ]:
os.environ['PW_RUN_SANDBOX'] = 'true'
pw.run()